# Expected Completion

In [60]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist #function to create adjacency matrix
import math
import tensorflow
import spektral
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from spektral.data import Dataset, BatchLoader, Graph, DisjointLoader
from spektral.transforms.normalize_adj import NormalizeAdj
from tensorflow.keras.layers import Dropout
from keras.callbacks import Callback, EarlyStopping
from spektral.layers import GCNConv, GlobalSumPool

In [37]:
plays = pd.read_csv("data/pass_receiver.csv")
pff = pd.read_csv("data/pffScoutingData.csv")


read in weekly tracking files and concatenate into one tracking file, then delete weeklies

In [38]:
week1 = pd.read_csv("data/week1.csv")
week2 = pd.read_csv("data/week2.csv")
week3 = pd.read_csv("data/week3.csv")
week4 = pd.read_csv("data/week4.csv")
week5 = pd.read_csv("data/week5.csv")
week6 = pd.read_csv("data/week6.csv")
week7 = pd.read_csv("data/week7.csv")
week8 = pd.read_csv("data/week8.csv")
tracking = pd.concat([week1,week2,week3,week4,week5,week6,week7,week8])


In [39]:
#save memory
del week1,week2,week3,week4,week5,week6,week7,week8

In [40]:
tracking.columns

Index(['gameId', 'playId', 'nflId', 'frameId', 'time', 'jerseyNumber', 'team',
       'playDirection', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event'],
      dtype='object')

add unique identifier of each play and frame (some playIds repeat over games)

In [41]:
tracking["comb_id"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str")
pff["comb_id"] = pff["gameId"].astype("str")+" "+pff["playId"].astype("str")
plays["comb_id"] = plays["gameId"].astype("str")+" "+plays["playId"].astype("str")
tracking["comb_and_frame"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str")+" "+tracking["frameId"].astype("str")

isolate frames at time of pass

In [42]:
#isolate just forward passes and leave out a certain frame (duplicate forward pass for a play)
just_forward_passes = tracking[(tracking["event"]=="pass_forward") & (tracking["comb_and_frame"]!="2021091203 3740 32")& (tracking["team"]!="football")]
#bring in receiver info from plays
tracking_plays =just_forward_passes.merge(plays[["comb_id","possessionTeam","is_pass_failed","PassReceiver_nflId"]], how="left",on="comb_id" )
#and qb info from pff
tracking_merged_all = tracking_plays.merge(pff[["comb_id","nflId","pff_role"]], how = "left",on=["comb_id","nflId"])

In [43]:
len(just_forward_passes)

166034

In [44]:
len(tracking_plays)

166276

isolate tracking to just QB, OL, receiver and defense

In [45]:
tracking_players_filtered = tracking_merged_all[(tracking_merged_all["nflId"]==tracking_merged_all["PassReceiver_nflId"])|(tracking_merged_all["pff_role"]=="Pass")|(tracking_merged_all["pff_role"]=="Pass Block")|(tracking_merged_all["team"]!=tracking_merged_all["possessionTeam"])]

# Function to take in 1 pass play and put it in graph format( list containing adjacency matrix, features, y) this will then be parallelized for all passes in the data

In [46]:
categories = ["Coverage","Pass","Pass Block","Pass Rush", "Pass Route"]

In [47]:
def angle_relative_diff(a,b):
    diff_list = []
    for i in range(len(a)):
        difference = a[i] - b[0]
        if difference < -180:
            difference +=360
        elif difference > 180:
            difference -=360
        diff_list.append(difference)
    return diff_list

In [48]:
def one_play_graph_function(data):
    ##one_play_formatted_list = []
    data = data.reset_index(drop=True)
    #get y, the response variable for this play (binary pass completed or not) and possesionteam (for CV)
    y = data["is_pass_failed"][0]
    team = data["possessionTeam"][0]
    #put play in adjacency matrix format, the distance of all players to each other (this will then be filtered to just link relevant players to QB or WR)
    data_dist = data[["x","y"]]
    adj = cdist(data_dist.values, data_dist.values)
    for index,row in data.iterrows():
        #connect QB to pass rush/block and wr, WR to coverage and QB, remove other connections by setting to zero
        if (row["pff_role"]=="Coverage" )| (row["pff_role"]=="Pass Route"):
            condition = ((data["pff_role"]=="Pass Block")|(data["pff_role"]=="Pass Rush"))
        else:
            condition = ((data["pff_role"]=="Coverage"))
        adj[index][condition] = 0
    #then put play in features format, getting a table containing position, speed, orientation etc
    data["onPossessionTeam"] = np.where(data["possessionTeam"]==data["team"],1,0)
    QB= data[data["pff_role"]=="Pass"]
    data["rel_o"] = data["o"]-QB["o"].iloc[0]
    data["rel_dir"] = data["dir"]-QB["dir"].iloc[0]
    data = data[["onPossessionTeam","s","rel_dir","rel_o","pff_role"]]
    one_hot = pd.get_dummies(data['pff_role'].astype(pd.CategoricalDtype(categories=categories)))
    data= data.drop('pff_role',axis = 1)
    # Join the encoded df
    data = data.join(one_hot)
    one_play_formatted_list = {"team":team,"y":y,"graph_a":adj,"graph_x":data}
    return one_play_formatted_list

In [49]:
graph_list = []
for i in range(len(tracking_players_filtered.comb_id.unique())):
#for i in range(10):
    
    one_play = tracking_players_filtered[tracking_players_filtered.comb_id==tracking_players_filtered.comb_id.unique()[i]]
    #print(one_game.comb_id.unique()[i])
    one_play = one_play.reset_index(drop=True)
    result = one_play_graph_function(one_play)
    graph_list.append(result.copy())


get list of indexes of possession teams to allow masking for Cross Validation

In [50]:
team_list = []
for i in range(len(graph_list)):
    team_list.append(graph_list[i]["team"])
unique_team_list = list(set(team_list))

# GNN Functions

In [61]:
#variables
learning_rate = 1e-3  # Learning rate
epochs = 100  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 21  # Batch size
callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=0)]

In [52]:
graph_list[1]["graph_x"]

,onPossessionTeam,s,rel_dir,rel_o,Coverage,Pass,Pass Block,Pass Rush,Pass Route
0,0,1.05,-99.68,-131.26,0,0,0,1,0
1,0,0.55,50.45,-149.54,0,0,0,1,0
2,1,3.26,-81.74,-225.26,0,0,1,0,0
3,0,4.49,40.53,-158.64,1,0,0,0,0
4,0,1.08,-27.40,-84.06,0,0,0,1,0
5,0,6.04,-162.82,-186.24,1,0,0,0,0
6,1,8.09,78.49,91.14,0,0,0,0,1
7,1,1.30,-53.38,58.17,0,0,1,0,0
8,1,0.22,0.00,0.00,0,1,0,0,0
9,0,3.08,-12.34,-144.25,0,0,0,1,0


put data in graph format to feed in to GNN

In [53]:
class GraphDataset(Dataset):
    def __init__(self, n_samples, df, n_outcome=1, **kwargs):
        self.n_samples = n_samples
        self.df = df  
        self.n_outcome = n_outcome
        super().__init__(**kwargs)

    def read(self):
        output = []
        for i in range(self.n_samples):
            # Node features
            iter_x = self.df[i]["graph_x"].copy()
            #print(iter_x)
            x = np.array(iter_x)#.reshape(len(iter_x),9)

            # Edges
            iter_a =  self.df[i]["graph_a"].copy()
            the_length = len(iter_a)
            a = np.array(iter_a).reshape(the_length,the_length)
            #print(iter_a)
            y = int(self.df[i]["y"])
            
           
            output.append(Graph(x=x, a=a, y=y))
        return(output)

train xCompletion model

In [54]:
class xCompletion(Model):

    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.5)
        self.dense = Dense(n_labels, 'sigmoid')

    def call(self, inputs):
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dense(out)

        return out

Cross Validation with k = 32 by loop through and for each team use as val (remove from training) and store predictions

In [62]:
prediction_list = []
for i in range(len(unique_team_list)):
    team_in = [j for j, e in enumerate(team_list) if e == unique_team_list[i]]
    team_out = [j for j, e in enumerate(team_list) if e != unique_team_list[i]]
    training_data = [graph_list[i] for i in team_out]
    val_data = [graph_list[i] for i in team_in]
    all_data = graph_list
    len_train = len(training_data)
    len_val = len(val_data)
    len_all = len(graph_list)
    data_tr = GraphDataset(n_samples = len_train, df = training_data, transforms=NormalizeAdj())
    data_va = GraphDataset(n_samples = len_val, df = val_data, transforms=NormalizeAdj())
    data_all = GraphDataset(n_samples = len_all, df=graph_list, transforms=NormalizeAdj())

# Data loaders
    loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
    loader_va = BatchLoader(data_va, batch_size=batch_size)
    loader_all = BatchLoader(data_all, batch_size=batch_size)

    model = xCompletion(200, 1)
    model.compile('adam', "binary_crossentropy","binary_accuracy")

    model.fit(loader_tr.load(), validation_data= loader_va.load(), steps_per_epoch=loader_tr.steps_per_epoch,
    validation_steps=loader_va.steps_per_epoch, epochs=100,
      callbacks=callbacks)
    predictions = model.predict(loader_all.load(), steps =loader_all.steps_per_epoch)
    prediction_list.append(predictions)
    

Epoch 1/100
347/347 [==============================] - 1s 3ms/step - loss: 31.7328 - binary_accuracy: 0.5204 - val_loss: 8.2136 - val_binary_accuracy: 0.6176
Epoch 2/100
347/347 [==============================] - 1s 3ms/step - loss: 23.4201 - binary_accuracy: 0.5185 - val_loss: 14.7380 - val_binary_accuracy: 0.5110
Epoch 3/100
347/347 [==============================] - 1s 3ms/step - loss: 18.4984 - binary_accuracy: 0.5200 - val_loss: 5.5022 - val_binary_accuracy: 0.6287
Epoch 4/100
347/347 [==============================] - 1s 3ms/step - loss: 13.4259 - binary_accuracy: 0.5298 - val_loss: 4.4221 - val_binary_accuracy: 0.6287
Epoch 5/100
347/347 [==============================] - 1s 3ms/step - loss: 10.0492 - binary_accuracy: 0.5300 - val_loss: 4.6031 - val_binary_accuracy: 0.5699
Epoch 6/100
347/347 [==============================] - 1s 3ms/step - loss: 7.4018 - binary_accuracy: 0.5358 - val_loss: 3.4858 - val_binary_accuracy: 0.5809
Epoch 7/100
347/347 [==============================]